In [1]:
import numpy as np
import pandas as pd
from pyecharts.charts import Bar
from pyecharts import options as opts
from src.describe import summary_cluster, one_hot_encode_clusters, random_forest_train, render_importance_rank

設定cluster csv路徑並讀取資料

In [2]:
cluster_path = './cluster-result/1713856215_som/cluster_140.csv'
cluster = pd.read_csv(cluster_path)

根據cluster進行劃分

In [3]:

chart = summary_cluster(cluster)
chart.render_notebook()

# Unsupervised to Supervised

將各分群的資料獨立並進行binary encoding

In [4]:
one_hot_encoded_clusters = one_hot_encode_clusters(cluster)

每個分群進行隨機森林建模 (Binary classification)

In [5]:
kappas = []
importance_bars = []
for cluster_index in range(len(one_hot_encoded_clusters)):
	feature_importance_df, kappa = random_forest_train(
		one_hot_encoded_clusters,
		cluster_index
	)
	importance_bar = render_importance_rank(
		feature_importance_df,
		cluster_index
	)
	kappas.append(kappa)
	importance_bars.append(importance_bar)


Cluster 0 | Accuracy: 0.99636 | Kappa: 0.9906882982369359
Cluster 1 | Accuracy: 0.9971466666666666 | Kappa: 0.9693446465301372
Cluster 2 | Accuracy: 0.9977066666666666 | Kappa: 0.9807310820459683
Cluster 3 | Accuracy: 0.9983066666666667 | Kappa: 0.9950298951415887
Cluster 4 | Accuracy: 0.9954 | Kappa: 0.9391822272336642
Cluster 5 | Accuracy: 0.9978 | Kappa: 0.7559344513501134
Cluster 6 | Accuracy: 0.9992 | Kappa: 0.6100384762036812
Cluster 7 | Accuracy: 0.99692 | Kappa: 0.9730948969400842
Cluster 8 | Accuracy: 0.9962533333333333 | Kappa: 0.903512899604245
Cluster 9 | Accuracy: 0.9982266666666667 | Kappa: 0.8435611461530659
Cluster 10 | Accuracy: 0.9979733333333334 | Kappa: 0.8979149431604732
Cluster 11 | Accuracy: 0.99816 | Kappa: 0.9781790822354745
Cluster 12 | Accuracy: 0.9965733333333333 | Kappa: 0.987728716573339
Cluster 13 | Accuracy: 0.9984533333333333 | Kappa: 0.9452623982838516
Cluster 14 | Accuracy: 0.9986133333333334 | Kappa: 0.9484662873389001
Cluster 15 | Accuracy: 0.99816 

In [6]:


xy_pairs = list(zip(
	[cluster_index for cluster_index in range(len(one_hot_encoded_clusters))], 
	kappas
))
sorted_pairs = sorted(xy_pairs, key=lambda pair: pair[1])
xaxis_sorted, yaxis_sorted = zip(*sorted_pairs)
chart = Bar()
chart.add_xaxis(xaxis_sorted)
chart.add_yaxis("Kappa", yaxis_sorted)
chart.set_global_opts(
	title_opts=opts.TitleOpts(title="Cluster Random Forest Kappa")
)


In [19]:

importance_bars[10].render_notebook()